In [3]:
# Imports libraries
import os
import re
import time
import requests
import joblib
import pandas as pd
import numpy as np

# Utilities
from geopy.geocoders import Nominatim



In [6]:
df = pd.read_csv("../data/raw/maize_yield.csv")
df.head()

,REGION,DISTRICT,YEAR,CROP,YIELD
0,ASHANTI,Bekwai muni (Amansie East),2006,MAIZE,1.14
1,ASHANTI,Amansie Central,2006,MAIZE,1.13
2,ASHANTI,Amansie West,2006,MAIZE,1.05
3,ASHANTI,Ejura Sekyedumase,2006,MAIZE,1.22
4,ASHANTI,Sekyere West,2006,MAIZE,1.31


In [7]:
df.tail()

,REGION,DISTRICT,YEAR,CROP,YIELD
1676,WESTERN,Bibiani-Anhwiaso-Bekwai,2016,MAIZE,1.11
1677,WESTERN,Sefwi-Wiawso,2016,MAIZE,1.96
1678,WESTERN,Akontombra,2016,MAIZE,1.15
1679,WESTERN,Bia,2016,MAIZE,2.28
1680,WESTERN,Juabeso,2016,MAIZE,1.63


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1681 entries, 0 to 1680
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   REGION    1681 non-null   object 
 1   DISTRICT  1681 non-null   object 
 2   YEAR      1681 non-null   int64  
 3   CROP      1681 non-null   object 
 4   YIELD     1681 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 65.8+ KB


In [9]:

def clean_district_name(name):
    """Standardizes district names for consistent merging."""
    if pd.isna(name):
        return ""

    name = str(name).lower()

    # Remove text in brackets
    name = re.sub(r'\s*\([^)]*\)', '', name)

    # Remove common suffixes
    suffixes = [
        ' municipal', ' muni', ' district',
        ' metropolitan', ' metro'
    ]
    for suffix in suffixes:
        name = name.replace(suffix, '')

    # Normalize spacing and symbols
    name = name.replace('-', ' ')
    name = re.sub(r'\s+', ' ', name).strip()

    return name.title()


def get_soil_map():
    """Maps Ghana's administrative regions to their dominant soil types."""
    return {
        # Forest zones
        'Ashanti': 'Forest Ochrosol',
        'Brong Ahafo': 'Forest Ochrosol',
        'Bono': 'Forest Ochrosol',
        'Bono East': 'Forest Ochrosol',
        'Ahafo': 'Forest Ochrosol',
        'Eastern': 'Forest Ochrosol',
        'Western': 'Forest Ochrosol',
        'Western North': 'Forest Ochrosol',
        'Central': 'Forest Ochrosol',

        # Savanna zones
        'Northern': 'Savanna Ochrosol',
        'Savannah': 'Savanna Ochrosol',
        'North East': 'Savanna Ochrosol',
        'Upper East': 'Savanna Ochrosol',
        'Upper West': 'Savanna Ochrosol',
        'Oti': 'Savanna Ochrosol',

        # Coastal & special zones
        'Greater Accra': 'Coastal Savannah',
        'Volta': 'Tropical Black Earth'
    }


In [12]:
df_yield = pd.read_csv("../data/raw/maize_yield.csv")

# Clean column names
df_yield.columns = [c.strip().title() for c in df_yield.columns]

# Standardize key columns
dist_col = next(c for c in df_yield.columns if 'District' in c)
df_yield.rename(columns={dist_col: 'District'}, inplace=True)

df_yield['District'] = df_yield['District'].apply(clean_district_name)

print(f"Yield Data Loaded: {len(df_yield)} rows (2010–2016)")


Yield Data Loaded: 1681 rows (2010–2016)


In [13]:
print("Imputing 2017–2021 data using National Averages...")

unique_districts = df_yield['District'].unique()

national_avgs = pd.DataFrame([
    {'Year': 2017, 'Yield': 2.04},
    {'Year': 2018, 'Yield': 2.25},
    {'Year': 2019, 'Yield': 2.53},
    {'Year': 2020, 'Yield': 2.58},
    {'Year': 2021, 'Yield': 2.53}
])

new_rows = (
    pd.MultiIndex.from_product(
        [unique_districts, national_avgs['Year']],
        names=['District', 'Year']
    )
    .to_frame(index=False)
    .merge(national_avgs, on='Year')
)



Imputing 2017–2021 data using National Averages...


In [14]:
new_rows.head()

,District,Year,Yield
0,Bekwai,2017,2.04
1,Bekwai,2018,2.25
2,Bekwai,2019,2.53
3,Bekwai,2020,2.58
4,Bekwai,2021,2.53


In [15]:
new_rows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   District  1005 non-null   object 
 1   Year      1005 non-null   int64  
 2   Yield     1005 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 23.7+ KB


In [16]:
df_extended = pd.concat(
    [df_yield[['District', 'Year', 'Yield']], new_rows],
    ignore_index=True
)

In [17]:
df_extended.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686 entries, 0 to 2685
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   District  2686 non-null   object 
 1   Year      2686 non-null   int64  
 2   Yield     2686 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 63.1+ KB


In [18]:
weather_path = "../data/processed/Weather_Data.csv"

print("Downloading weather data from NASA POWER API...")

geolocator = Nominatim(user_agent="ghana_maize_project_v1")
base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"

weather_data = []
  
for district in unique_districts:
    try:
        # Geolocate
        loc = geolocator.geocode(f"{district}, Ghana", timeout=10)
        if loc:
            # API Request
            params = {
                "parameters": "T2M,PRECTOTCORR,RH2M,ALLSKY_SFC_SW_DWN,GWETTOP",
                "community": "AG", "longitude": loc.longitude, "latitude": loc.latitude,
                "start": "20100101", "end": "20211231", "format": "JSON"
            }
            resp = requests.get(base_url, params=params)
            
            if resp.status_code == 200:
                d = resp.json()['properties']['parameter']
                temp_df = pd.DataFrame({
                    'Date': pd.to_datetime(list(d['T2M'].keys())),
                    'Rainfall': list(d['PRECTOTCORR'].values()),
                    'Temperature': list(d['T2M'].values()),
                    'Humidity': list(d['RH2M'].values()),
                    'Sunlight': list(d['ALLSKY_SFC_SW_DWN'].values()),
                    'Soil_Moisture': list(d['GWETTOP'].values()),
                    'District': district
                })
                weather_data.append(temp_df)
        time.sleep(1) # Be polite to API
    except Exception as e:
        print(f"Error for {district}: {e}")
        continue

df_weather = pd.concat(weather_data, ignore_index=True)
df_weather.to_csv(weather_path, index=False)

print("Weather Download Complete.")


Error for Sawla/Tuna/Kalba: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Weather Download Complete.


In [19]:
df_weather.head()

,Date,Rainfall,Temperature,Humidity,Sunlight,Soil_Moisture,District
0,2010-01-01,0.04,27.66,70.27,16.06,0.49,Bekwai
1,2010-01-02,0.08,27.92,67.54,16.74,0.48,Bekwai
2,2010-01-03,0.08,28.33,61.83,17.21,0.48,Bekwai
3,2010-01-04,0.10,27.69,62.06,18.59,0.48,Bekwai
4,2010-01-05,0.17,27.80,58.23,17.12,0.48,Bekwai


In [33]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793323 entries, 0 to 793322
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           793323 non-null  datetime64[ns]
 1   Rainfall       793323 non-null  float64       
 2   Temperature    793323 non-null  float64       
 3   Humidity       793323 non-null  float64       
 4   Sunlight       793323 non-null  float64       
 5   Soil_Moisture  793323 non-null  float64       
 6   District       793323 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 42.4+ MB


In [41]:
df_weather["Date"] = pd.to_datetime(df_weather["Date"])
df_weather["Year"] = df_weather["Date"].dt.year
df_weather["Month"] = df_weather["Date"].dt.month

season_weather = df_weather[df_weather["Month"].between(4, 8)]

weather_agg = (
    season_weather
    .groupby(["District", "Year"], as_index=False)
    .agg({
        "Rainfall": "sum",
        "Temperature": "mean",
        "Humidity": "mean",
        "Sunlight": "mean",
        "Soil_Moisture": "mean"
    })
)


In [42]:
weather_agg.columns = [
    "District", "Year",
    "Rainfall",
    "Temperature",
    "Humidity",
    "Sunlight",
    "Soil_Moisture"
]


In [43]:
weather_agg.head()

,District,Year,Rainfall,Temperature,Humidity,Sunlight,Soil_Moisture
0,A.M.A,2010,647.58,26.411634,86.390588,16.427908,0.809216
1,A.M.A,2011,670.85,25.852745,87.471373,15.939739,0.827974
2,A.M.A,2012,683.60,25.591503,86.877582,16.625163,0.814248
3,A.M.A,2013,481.38,25.802222,85.133856,16.894248,0.761242
4,A.M.A,2014,596.92,26.129346,85.771699,16.374771,0.782484


In [44]:
df_soil = (
    df_yield
    .assign(
        Region=lambda x: x.get("Region", pd.NA)
    )
    [["District", "Region"]]
    .drop_duplicates()
    .assign(
        Region=lambda x: x["Region"].str.title().str.strip(),
        Soil_Type=lambda x: (
            x["Region"]
            .map(get_soil_map())
            .fillna("Forest Ochrosol")
        )
    )
    [["District", "Soil_Type"]]
)


In [39]:
df_soil.head()

,District,Soil_Type
0,Bekwai,Forest Ochrosol
1,Amansie Central,Forest Ochrosol
2,Amansie West,Forest Ochrosol
3,Ejura Sekyedumase,Forest Ochrosol
4,Sekyere West,Forest Ochrosol


In [45]:
# -------------------------
# Merge datasets
# -------------------------
df = (
    df_extended
    .merge(weather_agg, on=["District", "Year"], how="inner")
    .merge(df_soil[["District", "Soil_Type"]], on="District", how="left")
)

# -------------------------
# Yield winsorization
# -------------------------
YIELD_CAP = 4.0
df["Yield"] = df["Yield"].clip(upper=YIELD_CAP)

# -------------------------
# Event shock indicators
# -------------------------
df = df.assign(
    Pest_Risk=lambda x: x["Year"].isin([2016, 2017]).astype(int),
    PFJ_Policy=lambda x: (x["Year"] >= 2017).astype(int)
)

# -------------------------
# Lag features
# -------------------------
df = (
    df.sort_values(["District", "Year"])
      .assign(Yield_Lag1=lambda x: x.groupby("District")["Yield"].shift(1))
      .dropna()
      .reset_index(drop=True)
)

# -------------------------
# Save master dataset
# -------------------------
final_path = "../data/final/Ghana_Maize_Data.csv"
df.to_csv(final_path, index=False)

print(f"Final Dataset Created: {df.shape}")


Final Dataset Created: (1764, 12)
